# Setup

In [11]:
import os
import sys
from glob import glob

import cartopy.crs as ccrs
import numpy as np
import xarray as xr

# Local Utils
sys.path.insert(0, "/glade/work/zespinosa/Projects/climate-utils")
from utils import  (
    plot_stationary_sp,
    xarray_time_to_monthly,
    from_pickle,
)

In [2]:
"""
Note to self: when scaling memory, we also need to scale the number of processes and the size of the cluster. 
Dask errors are not helpful with this error, so be diligent.
"""
# client.shutdown()
# from dask_jobqueue import PBSCluster
# cluster = PBSCluster(
    # Job scheduler specific keywords
    # project="UWAS0118",
    # walltime="06:00:00",
    # queue="economy",
    # local_directory="/glade/scratch/zespinosa/",
    # Dask-worker specific keyworkds
    # processes=32,  # 32 Number of Python processes to cut up each job
    # memory="16", # "16"
# )
# cluster.scale(4)
from dask.distributed import Client
client = Client()
client

2023-04-04 14:07:54,609 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-5nvodw7f', purging
2023-04-04 14:07:54,610 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-lq_lw9ce', purging
2023-04-04 14:07:54,610 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-admidkx6', purging
2023-04-04 14:07:54,611 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-clgb4jf9', purging
2023-04-04 14:07:54,611 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-yfz8sgqb', purging
2023-04-04 14:07:54,612 - distributed.diskutils - INFO - Found stale lock file and directory '/glade/scratch/zespinosa/dask-worker-space/worker-dmhv3fom', purging
2023-04-04 14:07:54,61

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 9
Total threads: 72,Total memory: 251.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33263,Workers: 9
Dashboard: /proxy/8787/status,Total threads: 72
Started: Just now,Total memory: 251.47 GiB
Comm: tcp://127.0.0.1:34267,Total threads: 8
Dashboard: /proxy/44627/status,Memory: 27.94 GiB
Nanny: tcp://127.0.0.1:34781,


# UTILS

In [3]:
def detrend_data(data, x, x_dim, deg=1):
    """
    Detrend data using n-degree least squares fit

    Arguments:
    -----------
        data [Dataset, DataArray](..., x_dim): data to detrend (y)
        x [DataArray](x_dim): dimension to detrend along (x)
        x_dim ([tr]: name of dimension along which to detrend
        deg [int]: degree of polynomial to fit

    Returns:
    --------
        da [Dataset](..., sia, sie): detrended data
    """
    results = data.polyfit(dim=x_dim, skipna=True, deg=deg)
    new_data = data - xr.polyval(x, results.polyfit_coefficients)
    da = xr.DataArray(new_data, coords=data.coords, dims=data.dims, attrs=data.attrs)
    return da

CESM-NUDGED DAILY ATM DATA FOR STORMS (Grab entire month of Nov. and Dec.): 
1. November 14 and Novomber 21 (Ross Sea)
2. December 21 (Weddell Sea)
VARS: 
- siconc
- 2m temp
- mslp
- 10m wind vectors

# CESM-NUDGED SEA ICE THERMO and DYNAMIC TENDENCIES

In [9]:
def load_monthly_tendency_data(futdir, histdir, myvariables , start_year=1985):
    # Get Load Files
    dfiles = sorted(
        glob(os.path.join(histdir, "*.h.*"))
        + glob(  # monthly data files 1950-2005
            os.path.join(futdir, "*.h.*")
        )  # monthly data files 2006 - 2024
    )

    drop_vars = xr.open_dataset(dfiles[0]).drop_vars(myvariables).data_vars

    dfiles = [
        f for f in dfiles if int(f.split(".")[-2][:4]) >= start_year
    ]  # remove files before start year
    
    ds = xr.open_mfdataset(
        dfiles,
        drop_variables=drop_vars, # this is absolutely essential (10x faster)
        chunks="auto",
        parallel=True,
        coords="minimal",
        data_vars=myvariables,
    )
    ds = ds.convert_calendar("standard")
    ds["time"] = np.arange("1985-01", "2025-01", dtype="datetime64[M]") # correct 1 month error

    return ds


def process_tendencies():
    myvariables=["daidtd", "daidtt"]
    
    ds_dt = load_monthly_tendency_data(
        futdir="/glade/scratch/wriggles/archive/nudge_era_1950_ens01_21C/ice/hist",
        histdir="/glade/scratch/wriggles/archive/nudge_era_1950_ens01/ice/hist",
        # myvariables=["dvidtd", "dvidtt", "daidtd", "daidtt"]
        myvariables=myvariables,
    )
    ds_dt = ds_dt.rename({"nj": "longitude", "ni": "latitude"})
    # Fix lat/lon data
    pop_areacello = xr.open_dataset("/glade/work/zespinosa/GRIDS/areacello_fx_CESM1-CAM5_historical_r0i0p0.nc")
    ds_dt.TLON[:] = pop_areacello.areacello.lon.data
    ds_dt.TLAT[:] = pop_areacello.areacello.lat.data
    for cvar in myvariables:
        print(f"starting {cvar}")
        ds_monthly = xarray_time_to_monthly(ds_dt[cvar])
        ds_anoms = detrend_data(ds_monthly, ds_monthly.year, "year", deg=1)
        ds_anoms.to_netcdf(f"/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_{cvar}_anoms_198501-202412.nc")
        ds_monthly.to_netcdf(f"/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_{cvar}_198501-202412.nc")

process_tendencies()

starting daidtd
starting daidtt


# CESM-NUDGED WINDS and SSTS - process ssts

In [ ]:
loadPath = list(sorted(glob("/glade/scratch/wriggles/archive/nudge_era_1950_ens01_21C/ocn/hist/*")))
loadPath = [fp for fp in loadPath if ("2022" not in fp) and ("2023" not in fp) and ("2024" not in fp)]
loadPath = np.concatenate([loadPath, list(sorted(glob("/glade/scratch/wriggles/archive/fcast_30deg_sst_01/ocn/hist/*")))])
drop_vars = from_pickle(
    "/glade/work/zespinosa/Projects/Antarctica_2022/cesm_nudged/drop_vars.pkl"
)
aice = xr.open_mfdataset(
    loadPath[:],
    parallel=True,
    chunks=int(1e4),
    drop_variables=drop_vars,
)

# CESM-NUDGED WINDS and SSTS - process sea ice 

In [14]:
"""
Extract variables that we are interested in, combine monthly files, and save to new netcdf. TODO: adapt this once we have full data from Ed
"""
loadPath = list(sorted(glob("/glade/scratch/wriggles/archive/nudge_era_1950_ens01_21C/ice/hist/*")))
loadPath = [fp for fp in loadPath if ("2022" not in fp) and ("2023" not in fp) and ("2024" not in fp)]
loadPath = np.concatenate([loadPath, list(sorted(glob("/glade/scratch/wriggles/archive/fcast_30deg_sst_01/ice/hist/*")))])
drop_vars = from_pickle(
    "/glade/work/zespinosa/Projects/Antarctica_2022/cesm_nudged/drop_vars.pkl"
)
print("loading aice")
aice = xr.open_mfdataset(
    loadPath[:],
    parallel=True,
    chunks=int(1e4),
    drop_variables=drop_vars,
)
print("loading aice past")
aice_past = xr.open_dataset("/glade/scratch/zespinosa/nudge_era_1950_ens01_195001-200512/nudge_era_1950_ens01.cice.h.aice.195001-200512.nc")

aice = xr.concat([aice_past.aice, aice.aice], dim="time")
print(aice)

aice["time"] = np.arange("1950-01", "2022-04", dtype="datetime64[M]")
siconcCESM = aice.sel(time=np.arange("1985-01", "2022-04", dtype="datetime64[M]"))
siconcCESM = siconcCESM.rename({"nj": "longitude", "ni": "latitude"})

# Fix lat/lon data
pop_areacello = xr.open_dataset("/glade/work/zespinosa/GRIDS/areacello_fx_CESM1-CAM5_historical_r0i0p0.nc")
siconcCESM.TLON[:] = pop_areacello.areacello.lon.data
siconcCESM.TLAT[:] = pop_areacello.areacello.lat.data
siconcCESM_monthly = xarray_time_to_monthly(siconcCESM)

# ####### SICONC CESM-Nudged Data #######
print("saving datafiles")
siconcCESM_anoms = detrend_data(siconcCESM_monthly, siconcCESM_monthly.year, "year", deg=1)
siconcCESM_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_sst-wind_nudged_siconc-anoms_198501-202412.nc")
siconcCESM_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_sst-wind_nudged_siconc_198501-202412.nc")


loading aice
loading aice past


2023-04-04 16:06:40,361 - distributed.nanny - WARNING - Restarting worker
2023-04-04 16:06:40,363 - distributed.nanny - WARNING - Restarting worker
2023-04-04 16:06:41,956 - distributed.nanny - WARNING - Restarting worker
2023-04-04 16:06:43,518 - distributed.nanny - WARNING - Restarting worker


<xarray.DataArray 'aice' (time: 867, nj: 384, ni: 320)>
dask.array<concatenate, shape=(867, 384, 320), dtype=float32, chunksize=(224, 192, 320), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1950-02-01 00:00:00 ... 2022-04-01 00:00:00
    TLON     (time, nj, ni) float32 320.6 321.7 322.8 323.9 ... nan nan nan nan
    TLAT     (time, nj, ni) float32 -79.22 -79.22 -79.22 -79.22 ... nan nan nan
    ULON     (time, nj, ni) float32 321.1 322.2 323.4 324.5 ... nan nan nan nan
    ULAT     (time, nj, ni) float32 -78.95 -78.95 -78.95 -78.95 ... nan nan nan
Dimensions without coordinates: nj, ni
Attributes:
    long_name:     ice area  (aggregate)
    units:         %
    comment:       none
    cell_methods:  time: mean
    time_rep:      averaged
saving datafiles


In [8]:
# Test Code
anoms = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_sst-wind_nudged_siconc-anoms_197901-202412.nc")
siconc = xr.open_dataarray("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc_197901-202412.nc")

# CESM-NUDGED OCEAN DATA

In [16]:
futdir = "/glade/scratch/wriggles/archive/nudge_era_1950_ens01_21C/ocn/hist"
histdir = "/glade/scratch/wriggles/archive/nudge_era_1950_ens01/ocn/hist"
# histdir = "/glade/campaign/univ/uwas0123/sosi_nudge_era/nudge_era_1950_ens01/ocn/hist"
myvariables = ["SST"]

def load_monthly_pop_data():
    drop_vars = xr.open_dataset(
        os.path.join(histdir, "nudge_era_1950_ens01.pop.h.1987-07.nc")
    ).drop_vars(myvariables)
    drop_vars = drop_vars.data_vars

    dfiles = sorted(
        glob(os.path.join(histdir, "*.h.*"))
        + glob(  # monthly data files 1950-2005
            os.path.join(futdir, "*.h.*")
        )  # monthly data files 2006 - 2024
    )
    dfiles = [
        f for f in dfiles if int(f.split(".")[-2][:4]) >= 1985
    ]  # remove files before 1985
    
    ds_ocn = xr.open_mfdataset(
        dfiles,
        drop_variables=drop_vars, # this is absolutely essential (10x faster)
        chunks="auto",
        parallel=True,
        coords="minimal",
        data_vars=myvariables,
    )
    ds_ocn = ds_ocn.convert_calendar("standard")
    ds_ocn["time"] = np.arange("1985-01", "2025-01", dtype="datetime64[M]") # correct 1 month error

    return ds_ocn


ds_pop = load_monthly_pop_data()
ds_pop_monthly = xarray_time_to_monthly(ds_pop.SST)
ds_pop_anoms = detrend_data(ds_pop_monthly, ds_pop_monthly.year, "year", deg=1)

# ds_pop_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_sst_anoms_198501-202412.nc")
# ds_pop_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_sst_198501-202412.nc")

# CESM-NUDGED ATMOS DATA

In [21]:
futdir = "/glade/scratch/wriggles/archive/nudge_era_1950_ens01_21C/atm/hist"
histdir = "/glade/scratch/wriggles/archive/nudge_era_1950_ens01/atm/hist"
savedir = "/glade/work/zespinosa/data/CESM_nudged/"
myvariables = ["PSL"]

def load_monthly_atm_data():
    drop_vars = xr.open_dataset(
        os.path.join(futdir, "nudge_era_1950_ens01_21C.cam.h0.2024-11.nc")
    ).drop_vars(myvariables)
    drop_vars = drop_vars.data_vars

    dfiles = sorted(
        glob(os.path.join(histdir, "*.h0*"))
        + glob(  # monthly data files 1950-2005
            os.path.join(futdir, "*.h0*")
        )  # monthly data files 2006 - 2024
    )
    dfiles = [
        f for f in dfiles if int(f.split(".")[-2][:4]) >= 1985
    ]  # remove files before 1985
    
    ds_atm = xr.open_mfdataset(
        dfiles,
        drop_variables=drop_vars, # this is absolutely essential (10x faster)
        chunks="auto",
        parallel=True,
        coords="minimal",
        data_vars=myvariables,
    )
    ds_atm = ds_atm.convert_calendar("standard")
    ds_atm["time"] = np.arange("1985-01", "2025-01", dtype="datetime64[M]") # correct 1 month error

    return ds_atm


ds_atm = load_monthly_atm_data()
ds_atm_monthly = xarray_time_to_monthly(ds_atm.PSL)
ds_atm_anoms = detrend_data(ds_atm_monthly, ds_atm_monthly.year, "year", deg=1)
ds_atm_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_psl_anoms_198501-202412.nc")
ds_atm_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm-nudged_psl_198501-202412.nc")

# CESM-NUDGED SICONC DATA

In [13]:
cesm_ice = xr.open_dataset(
    "/glade/work/zespinosa/Projects/Antarctica_2022/cesm_nudged/ens_01.nc", chunks={"time": 1, "nj": -1, "ni": -1}
)
cesm_ice = cesm_ice.sel(time=slice("1985-01", "2024-12"))
####### SICONC CESM-Nudged Data #######
siconcCESM = cesm_ice.aice
siconcCESM = siconcCESM.rename({"nj": "longitude", "ni": "latitude"})
# Fix lat/lon data
pop_areacello = xr.open_dataset("/glade/work/zespinosa/GRIDS/areacello_fx_CESM1-CAM5_historical_r0i0p0.nc")
siconcCESM.TLON[:] = pop_areacello.areacello.lon.data
siconcCESM.TLAT[:] = pop_areacello.areacello.lat.data
siconcCESM_monthly = xarray_time_to_monthly(siconcCESM)
siconcCESM_anoms = detrend_data(siconcCESM_monthly, siconcCESM_monthly.year, "year", deg=1)
siconcCESM_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc-anoms_198501-202412_updated.nc")
siconcCESM_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/cesm_nudged_siconc_198501-202412_updated.nc")

# LOAD NSIDC MONTHLY DATA

In [8]:
# Timely data
siconcObs = xr.open_dataset(
    # "/glade/work/zespinosa/data/nsidc/processed/siconc_NSIDC_197901-202210.nc"
    "/glade/work/zespinosa/data/nsidc/processed/siconc_NSIDC_197901-202302.nc"
).cdr_seaice_conc_monthly
siObs = xr.open_dataset(
    # "/glade/work/zespinosa/data/nsidc/processed/sia_sie_NSIDC_197901-202210.nc"
    "/glade/work/zespinosa/data/nsidc/processed/sia_sie_NSIDC_197901-202302.nc"
)

# Monthly data
siconcObs_monthly = xarray_time_to_monthly(siconcObs)
siObs_monthly = xarray_time_to_monthly(siObs)

siconcObs_anoms = detrend_data(siconcObs_monthly, siconcObs_monthly.year, "year", deg=1)
# siconcObs_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/nsidc_siconc-anoms_197901-202211.nc")
siconcObs_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/nsidc_siconc-anoms_197901-202302.nc")

sieObs_anoms = detrend_data(siObs_monthly.sie, siObs_monthly.year, "year", deg=1)
siconcObs_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/nsidc_sia-sie-anoms_197901-202302.nc")

NSIDC DAILY DATA FOR STORMS (Grab entire month of Nov. and Dec.): 
1. November 14 and Novomber 21 (Ross Sea)
2. December 21 (Weddell Sea)
siconc

# ERA5 Monthly SLP Data

In [66]:
eradir = "/glade/work/zespinosa/data/era5/monthly/MSLP"
dfiles = [os.path.join(eradir, "slp_single_level_1979_2021.nc"), os.path.join(eradir, "slp_single_level_2022.nc")]
era5_slp = xr.open_mfdataset(
    dfiles, 
    parallel=True,
    chunks={"latitude": -1, "longitude": -1, "time": 1}
).sel(expver=1)
era5_slp_monthly = xarray_time_to_monthly(era5_slp.sp)
era5_slp_anoms = detrend_data(era5_slp_monthly, era5_slp_monthly.year, "year", deg=1)
era5_slp_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_slp-monthly_197901-202211.nc")
era5_slp_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_slp-anoms_197901-202211.nc")

# ERA5 Monthly SST Data

In [12]:
eradir = "/glade/work/zespinosa/data/era5/monthly/SST"
dfiles = [os.path.join(eradir, "sst_single_level_1979_2021.nc"), os.path.join(eradir, "sst_single_level_2022.nc")]
era5_sst = xr.open_mfdataset(
    dfiles, 
    parallel=True,
    chunks={"latitude": -1, "longitude": -1, "time": 1}
).sel(expver=1)
era5_sst_monthly = xarray_time_to_monthly(era5_sst.sst)
era5_sst_anoms = detrend_data(era5_sst_monthly, era5_sst_monthly.year, "year", deg=1)
era5_sst_monthly.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_sst-monthly_197901-202211.nc")
era5_sst_anoms.to_netcdf("/glade/work/zespinosa/Projects/antarctic-2022_record-low_nudge-analysis/processed_data/era5_sst-anoms_197901-202211.nc")

ERA5 DAILY ATM DATA FOR STORMS (Grab entire month of Nov. and Dec.): 
1. November 14 and Novomber 21 (Ross Sea)
2. December 21 (Weddell Sea)
VARS: 
- 2m temp
- mslp
- 10m wind vectors

Use these spatial plots for sanity checks

In [ ]:
def spatial_plot(siconc, title, lon, lat, levels=np.arange(-100, 110, 10), cmap="RdBu"):
    fig, ax = plot_stationary_sp()
    img = ax.contourf(
        lon,
        lat,
        siconc,
        transform=ccrs.PlateCarree(), 
        levels=levels,
        cmap=cmap,
    )
    cbar2 = fig.colorbar(img, ax=ax)
    ax.set_title(title)
    fig.set_size_inches(6, 6)

In [28]:
def spatial_plot_atm(siconc, title, lon, lat, levels=np.arange(-14, 16, 2), cmap="RdBu"):
    fig, ax = plot_stationary_sp()
    img = ax.contour(
        lon,
        lat,
        siconc,
        transform=ccrs.PlateCarree(), 
        levels=levels,
        colors=["black"],
        negative_linestyle="dashed",
    )
    ax.clabel(img, img.levels, inline=True, fontsize=14)
    ax.set_title(title)
    fig.set_size_inches(6, 6)